In [2]:

import string
import re


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import pandas as pd
import numpy as np
#to preprocess tweets
import nltk
import string
import unicodedata
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

ps = nltk.PorterStemmer()
wn = WordNetLemmatizer()
stop = stopwords.words('english')
exclude = set(string.punctuation) 

words = nltk.download('words')

from bs4 import BeautifulSoup
from textblob import TextBlob

import contractions
import unicodedata
from unidecode import unidecode
from pandas_gbq import to_gbq
import os
from google.cloud import bigquery_storage
from google.cloud import bigquery

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [3]:
def read_data(path_to_csv_file):
    '''
    Reads csv files from specified paths

    Parameters:
            path to csv file locations
    Returns:
            dataframes of imported csv files
    '''
    # Read CSV files
    df = pd.read_csv(path_to_csv_file)
    
    return df

# call the function to import train data - update with relevant paths
df_user_last_game_played = read_data('last_played_game.csv')
df_all_available_games = read_data('all_games.csv')

In [4]:
df_user_last_game_played.head()

,event_date,event_timestamp,user_id,card_id,game,total_plays,ga_session_id,sessions,app_screen,session_duration_min,session_duration_sec,event_name,favourites,unfavourites,rank
0,2024-05-26,1716696869596000,201003768024,7461,AirPlane Survival,1,1.716697e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1
1,2024-05-14,1715670603189000,201147258789,25865,High School Gossip,1,1.715670e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1
2,2024-05-22,1716385795027000,201147454676,26075,Santa Airlines,1,1.716386e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1
3,2024-05-16,1715880685145000,2250170190602,187160,Connectricity,1,1.715879e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1
4,2024-05-20,1716239188475000,2250500253837,25905,Megacity Hop,1,1.716238e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1


In [5]:
df_user_last_game_played['event_name'].unique()

array(['gamesOpen', 'clickGamesUnfavourite', 'gamesPlayButton',
       'clickCardLink', 'clickGamesFavourite', 'clickGamesPlayFavourite',
       'clickGamesUnfavouriteFullScreen', 'clickGamesFavouriteFullScreen'],
      dtype=object)

In [6]:
df_all_available_games.head()

,game_title,game_id
0,Alewa,160698
1,Puzzle Scout,264279
2,Sweave,264278
3,Connectricity,187160
4,Soja Man,187163


In [7]:
class NltkPreprocessingSteps:
    
    
    
    def __init__(self, X):
        
        self.X = X


        self.sw_nltk = stopwords.words('english')
        new_stopwords = ['<*>','Ayoba','ayoba']
        self.sw_nltk.extend(new_stopwords)
        #self.sw_nltk.remove('not')
        '''
        self.pos_tag_dict = {"J": wordnet.ADJ,
                        "N": wordnet.NOUN,
                        "V": wordnet.VERB,
                        "R": wordnet.ADV}
        '''

        # '!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~' 32 punctuations in python
        # we dont want to replace . first time around
        self.remove_punctuations = string.punctuation.replace('.','')

    def remove_html_tags(self):
        self.X = self.X.apply(
                lambda x: BeautifulSoup(x, 'html.parser').get_text())
        return self


    def remove_accented_chars(self):
        self.X = self.X.apply(
                lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
        return self

    def replace_diacritics(self):
        self.X = self.X.apply(
                lambda x: unidecode(x, errors="preserve"))
        return self

    def to_lower(self):
        self.X = self.X.apply(lambda x: " ".join([word.lower() for word in x.split() if word and word not in self.sw_nltk]) if x else '')
        return self

    def expand_contractions(self):
        self.X = self.X.apply(
                lambda x: " ".join([contractions.fix(expanded_word) 
                            for expanded_word in x.split()]))
        return self

    def remove_numbers(self):
        self.X = self.X.apply(lambda x: re.sub(r'\d+', '', x))
        return self
    
    def remove_http(self):
        self.X = self.X.apply(lambda x: re.sub(r'http\S+', '', x))
        return self
                                               
    def remove_words_with_numbers(self):
        self.X = self.X.apply(lambda x: re.sub(r'\w*\d\w*', '', x))
        return self
    
    def remove_digits(self):
        self.X = self.X.apply(lambda x: re.sub(r'[0-9]+', '', x))
        return self
    
    def remove_special_character(self):
        self.X = self.X.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]+', ' ', x))
        return self
    
    def remove_white_spaces(self):
        self.X = self.X.apply(lambda x: re.sub(r'\s+', ' ', x).strip())
        return self
    
    def remove_extra_newlines(self):
        self.X == self.X.apply(lambda x: re.sub(r'[\r|\n|\r\n]+', ' ', x))
        return self


    def replace_dots_with_spaces(self):
        self.X = self.X.apply(lambda x: re.sub("[.]", " ", x))
        return self

    def remove_punctuations_except_periods(self):
        self.X = self.X.apply(
                     lambda x: re.sub('[%s]' %
                      re.escape(self.remove_punctuations), '' , x))
        return self

    def remove_all_punctuations(self):
        self.X = self.X.apply(lambda x: re.sub('[%s]' %
                          re.escape(string.punctuation), '' , x))
        return self

    def remove_double_spaces(self):
        self.X = self.X.apply(lambda x: re.sub(' +', '  ', x))
        return self

    def fix_typos(self):
        self.X = self.X.apply(lambda x: str(TextBlob(x).correct()))
        return self

    def remove_stopwords(self):
        # remove stop words from token list in each colum
        self.X = self.X.apply(
            lambda x: " ".join([ word for word in x.split() 
                                if word not in self.sw_nltk]) )
        return self
    
    def remove_singleChar(self):
        # remove stop words from token list in each colum
        self.X = self.X.apply(
            lambda x: " ".join([ word for word in x.split() 
                                if len(word)>2]) )
        return self

    def lemmatize(self):
        lemmatizer = WordNetLemmatizer()
        self.X = self.X.apply(
            lambda x: " ".join([ wn.lemmatize(word) for word in x.split()]))
        
        return self

    def get_processed_text(self):
        return self.X
    

txt_preproc_all_games = NltkPreprocessingSteps(df_all_available_games['game_title'] )
txt_preproc_user_games = NltkPreprocessingSteps(df_user_last_game_played['game'] )

processed_text_all_games =   txt_preproc_all_games.to_lower().remove_html_tags().remove_accented_chars().replace_diacritics().expand_contractions().remove_numbers().remove_digits().remove_special_character().remove_white_spaces().remove_extra_newlines().replace_dots_with_spaces().remove_punctuations_except_periods().remove_words_with_numbers().remove_singleChar().remove_double_spaces().lemmatize().remove_stopwords().get_processed_text()


processed_text_all_users =   txt_preproc_user_games.to_lower().remove_html_tags().remove_accented_chars().replace_diacritics().expand_contractions().remove_numbers().remove_digits().remove_special_character().remove_white_spaces().remove_extra_newlines().replace_dots_with_spaces().remove_punctuations_except_periods().remove_words_with_numbers().remove_singleChar().remove_double_spaces().lemmatize().remove_stopwords().get_processed_text()

df_all_available_games['game_title_processed']=processed_text_all_games
df_user_last_game_played['game_processed']=processed_text_all_users


In [8]:
df_all_available_games.head()

,game_title,game_id,game_title_processed
0,Alewa,160698,alewa
1,Puzzle Scout,264279,puzzle scout
2,Sweave,264278,sweave
3,Connectricity,187160,connectricity
4,Soja Man,187163,soja man


In [9]:
df_user_last_game_played.head()

,event_date,event_timestamp,user_id,card_id,game,total_plays,ga_session_id,sessions,app_screen,session_duration_min,session_duration_sec,event_name,favourites,unfavourites,rank,game_processed
0,2024-05-26,1716696869596000,201003768024,7461,AirPlane Survival,1,1.716697e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1,airplane survival
1,2024-05-14,1715670603189000,201147258789,25865,High School Gossip,1,1.715670e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1,high school gossip
2,2024-05-22,1716385795027000,201147454676,26075,Santa Airlines,1,1.716386e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1,santa airline
3,2024-05-16,1715880685145000,2250170190602,187160,Connectricity,1,1.715879e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1,connectricity
4,2024-05-20,1716239188475000,2250500253837,25905,Megacity Hop,1,1.716238e+09,1,AiA,0.0,0.0,gamesOpen,0,0,1,megacity hop


In [10]:

# TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_all_available_games['game_title_processed'] )

# Compute similarity scores
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:

# Function to get recommendations based on content similarity
def get_content_based_recommendations(game_name, cosine_sim=cosine_sim, df_all_available_games=df_all_available_games):
    idx = df_all_available_games[df_all_available_games['game_title_processed'] == game_name].index
    if len(idx) == 0:
        print(f"No similar games found for '{game_name}'")
        return []
    else:
        idx = idx[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Top 10 similar games
        similar_games = [df_all_available_games['game_title_processed'][i[0]] for i in sim_scores]
        return similar_games


In [24]:
get_content_based_recommendations('drone pickup service', cosine_sim=cosine_sim, df_all_available_games=df_all_available_games)

['alewa',
 'puzzle scout',
 'sweave',
 'connectricity',
 'soja man',
 'akonta',
 'baselab game',
 'anti poacher xtreme',
 'bubble attack',
 'penalty challenge']

In [13]:
def get_recommendations():
    distinct_games = df_user_last_game_played['game_processed'].drop_duplicates().dropna()
    games_recommendations_df=''
    if distinct_games.empty:
        print("No distinct games found. Exiting recommendation process.")
    else:
        recommendations_list = []

        for game_name in distinct_games:

            recommendations = get_content_based_recommendations(game_name)
            print(f"Content-based recommendations similar to '{game_name}':")
            for rank, recommendation in enumerate(recommendations, start=1):
                recommendations_list.append({
                    'game_name': game_name,
                    'recommendation_game': recommendation,
                    'ranking': rank
                })

        # Create the DataFrame after the loop
        recommendations_df = pd.DataFrame(recommendations_list)

        # Display the DataFrame with recommendations for each game
        print("\nDataFrame with recommendations for each game:")
        games_recommendations_df =recommendations_df
        return games_recommendations_df  

In [14]:
get_recommendations()

Content-based recommendations similar to 'airplane survival':
Content-based recommendations similar to 'high school gossip':
Content-based recommendations similar to 'santa airline':
Content-based recommendations similar to 'connectricity':
Content-based recommendations similar to 'megacity hop':
Content-based recommendations similar to 'happy crayon':
Content-based recommendations similar to 'math bat':
Content-based recommendations similar to 'kill spy':
Content-based recommendations similar to 'classic solitaire':
Content-based recommendations similar to 'air hockey':
Content-based recommendations similar to 'dream body workout':
Content-based recommendations similar to 'tripoly':
Content-based recommendations similar to 'oddbods monster truck driving':
Content-based recommendations similar to 'extreme baseball':
Content-based recommendations similar to 'teddy bubble rescue':
Content-based recommendations similar to 'drone pickup service':
Content-based recommendations similar to 'c

,game_name,recommendation_game,ranking
0,airplane survival,alewa,1
1,airplane survival,puzzle scout,2
2,airplane survival,sweave,3
3,airplane survival,connectricity,4
4,airplane survival,soja man,5
...,...,...,...
2195,tmnt epic mutant mission,soja man,6
2196,tmnt epic mutant mission,akonta,7
2197,tmnt epic mutant mission,baselab game,8
2198,tmnt epic mutant mission,anti poacher xtreme,9


In [15]:
df_all_available_games.head()

,game_title,game_id,game_title_processed
0,Alewa,160698,alewa
1,Puzzle Scout,264279,puzzle scout
2,Sweave,264278,sweave
3,Connectricity,187160,connectricity
4,Soja Man,187163,soja man


In [16]:

def pre_process_recommedation(df= get_recommendations()):
    
    df_reco= df
    #get_recommendations()
    #df_reco = pd.merge( df_reco,)

    # Perform a left join
    df_merged_reco = pd.merge(df_reco, df_all_available_games, left_on='recommendation_game', right_on='game_title_processed', how='left')
    df_user_rec=df_user_last_game_played[['user_id','game_processed']]
    df_user_games_recommendations = pd.merge(df_user_rec, df_merged_reco, left_on='game_processed', right_on='game_name', how='left')
    #df_user_games_recommendations['game_id'] = df_user_games_recommendations['game_id'].astype(int)
    df_user_games_recommendations['ranking'].fillna(-1, inplace=True)
    df_user_games_recommendations['ranking'] = df_user_games_recommendations['ranking'].astype(int)
    df_user_games_recommendations['country']=''
    df_user_games_recommendations['city'] =''
    df_user_games_recommendations['recommendation_type']='games'
    df_user_games_recommendations['recommendation_activity']='user_activity'
    df_user_games_recommendations[['user_id','game_id','ranking','recommendation_type','recommendation_activity']]
    #This is rename to conform with the current production database, we have to create a new table for the activity recommendation later and update the api point

    df_user_games_recommendations.rename(columns={'game_id': 'GameID'}, inplace=True)
    df_user_games_recommendations.rename(columns={'ranking': 'City_Ranking'}, inplace=True)
    return df_user_games_recommendations[['user_id','country','city','GameID','City_Ranking','recommendation_type','recommendation_activity']]

Content-based recommendations similar to 'airplane survival':
Content-based recommendations similar to 'high school gossip':
Content-based recommendations similar to 'santa airline':
Content-based recommendations similar to 'connectricity':
Content-based recommendations similar to 'megacity hop':
Content-based recommendations similar to 'happy crayon':
Content-based recommendations similar to 'math bat':
Content-based recommendations similar to 'kill spy':
Content-based recommendations similar to 'classic solitaire':
Content-based recommendations similar to 'air hockey':
Content-based recommendations similar to 'dream body workout':
Content-based recommendations similar to 'tripoly':
Content-based recommendations similar to 'oddbods monster truck driving':
Content-based recommendations similar to 'extreme baseball':
Content-based recommendations similar to 'teddy bubble rescue':
Content-based recommendations similar to 'drone pickup service':
Content-based recommendations similar to 'c

In [17]:
df_merged_reco = pd.merge(get_recommendations(), df_all_available_games, left_on='recommendation_game', right_on='game_title_processed', how='left')
df_merged_reco.head()

Content-based recommendations similar to 'airplane survival':
Content-based recommendations similar to 'high school gossip':
Content-based recommendations similar to 'santa airline':
Content-based recommendations similar to 'connectricity':
Content-based recommendations similar to 'megacity hop':
Content-based recommendations similar to 'happy crayon':
Content-based recommendations similar to 'math bat':
Content-based recommendations similar to 'kill spy':
Content-based recommendations similar to 'classic solitaire':
Content-based recommendations similar to 'air hockey':
Content-based recommendations similar to 'dream body workout':
Content-based recommendations similar to 'tripoly':
Content-based recommendations similar to 'oddbods monster truck driving':
Content-based recommendations similar to 'extreme baseball':
Content-based recommendations similar to 'teddy bubble rescue':
Content-based recommendations similar to 'drone pickup service':
Content-based recommendations similar to 'c

,game_name,recommendation_game,ranking,game_title,game_id,game_title_processed
0,airplane survival,alewa,1,Alewa,160698,alewa
1,airplane survival,puzzle scout,2,Puzzle Scout,264279,puzzle scout
2,airplane survival,sweave,3,Sweave,264278,sweave
3,airplane survival,connectricity,4,Connectricity,187160,connectricity
4,airplane survival,soja man,5,Soja Man,187163,soja man


In [18]:
df_user_rec=df_user_last_game_played[['user_id','game_processed']]
df_user_rec.head()

,user_id,game_processed
0,201003768024,airplane survival
1,201147258789,high school gossip
2,201147454676,santa airline
3,2250170190602,connectricity
4,2250500253837,megacity hop


In [19]:
df_user_games_recommendations = pd.merge(df_user_rec, df_merged_reco, left_on='game_processed', right_on='game_name', how='left')
df_user_games_recommendations.head()

,user_id,game_processed,game_name,recommendation_game,ranking,game_title,game_id,game_title_processed
0,201003768024,airplane survival,airplane survival,alewa,1.0,Alewa,160698.0,alewa
1,201003768024,airplane survival,airplane survival,puzzle scout,2.0,Puzzle Scout,264279.0,puzzle scout
2,201003768024,airplane survival,airplane survival,sweave,3.0,Sweave,264278.0,sweave
3,201003768024,airplane survival,airplane survival,connectricity,4.0,Connectricity,187160.0,connectricity
4,201003768024,airplane survival,airplane survival,soja man,5.0,Soja Man,187163.0,soja man


In [20]:
#df_user_games_recommendations['game_id'] = df_user_games_recommendations['game_id'].astype(int)
df_user_games_recommendations['ranking'].fillna(-1, inplace=True)
df_user_games_recommendations['ranking'] = df_user_games_recommendations['ranking'].astype(int)
df_user_games_recommendations['country']=''
df_user_games_recommendations['city'] =''
df_user_games_recommendations['recommendation_type']='games'
df_user_games_recommendations['recommendation_activity']='user_activity'
df_user_games_recommendations[['user_id','game_id','ranking','recommendation_type','recommendation_activity']]

df_user_games_recommendations.rename(columns={'game_id': 'GameID'}, inplace=True)
df_user_games_recommendations.rename(columns={'ranking': 'City_Ranking'}, inplace=True)
df_user_games_recommendations.head()


/var/folders/__/7g992vcd0fs5t43310f6sl8m0000gn/T/ipykernel_5839/3569105907.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_user_games_recommendations['ranking'].fillna(-1, inplace=True)


,user_id,game_processed,game_name,recommendation_game,City_Ranking,game_title,GameID,game_title_processed,country,city,recommendation_type,recommendation_activity
0,201003768024,airplane survival,airplane survival,alewa,1,Alewa,160698.0,alewa,,,games,user_activity
1,201003768024,airplane survival,airplane survival,puzzle scout,2,Puzzle Scout,264279.0,puzzle scout,,,games,user_activity
2,201003768024,airplane survival,airplane survival,sweave,3,Sweave,264278.0,sweave,,,games,user_activity
3,201003768024,airplane survival,airplane survival,connectricity,4,Connectricity,187160.0,connectricity,,,games,user_activity
4,201003768024,airplane survival,airplane survival,soja man,5,Soja Man,187163.0,soja man,,,games,user_activity


In [21]:

# Set up Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/mncedisimncwabe/Documents/Personal_Google_Service_Account/bright-arc-328707-b5e2d782b48b.json'

# Initialize the BigQuery client
client = bigquery.Client()

#RECO DATA
df=pre_process_recommedation(df=get_recommendations())

def create_and_load_recommendations(df):
    # Define the table schema
    schema = [
        bigquery.SchemaField("user_id", "INTEGER"),
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("city", "STRING"),
        bigquery.SchemaField("GameID", "INTEGER"),
        bigquery.SchemaField("City_Ranking", "INTEGER"),
        bigquery.SchemaField("recommendation_type", "STRING"),
        bigquery.SchemaField("recommendation_activity", "STRING"),
    ]

    # Define table ID and configuration
    dataset_id = 'bright-arc-328707.ayoba' 
    table_id = f'{dataset_id}.rec_games_recommendations_activity_staging'
    table = bigquery.Table(table_id, schema=schema)

    # Drop the table if it exists
    try:
        client.delete_table(table)
        print(f"Dropped table {table_id}")
    except:
        pass  # Ignore if the table doesn't exist


    # Create the table
    # Set clustering fields
    table.clustering_fields = ["user_id"]

    # Create the table
    table = client.create_table(table, exists_ok=True)
    print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}") 
    # Load the DataFrame into the newly created table
    to_gbq(df, table_id, project_id=table.project, if_exists='append')
    print('Data loading done')


create_and_load_recommendations(df)

def manage_down_stream_update():
    # Define the SQL for the MERGE operation
    merge_sql = """
    MERGE `bright-arc-328707.ayoba.rec_card_recommendations` AS A
    USING (SELECT * FROM `bright-arc-328707.ayoba.rec_games_recommendations_activity_staging` 
            where GameId is not null or GameId !=-1)  AS B
    ON (A.user_id = B.user_id 
        AND A.GameID = B.GameID
        AND B.recommendation_type = B.recommendation_type
        AND B.recommendation_activity)
    WHEN MATCHED AND A.ranking != B.ranking THEN
      UPDATE SET 

        A.ranking = B.ranking,

    WHEN NOT MATCHED BY TARGET THEN
      INSERT (user_id, country, city, GameID, City_Ranking, recommendation_type, recommendation_activity)
      VALUES (B.user_id, B.country, B.city, B.GameID, B.City_Ranking, B.recommendation_type, B.recommendation_activity)
    WHEN NOT MATCHED BY SOURCE 
        AND recommendation_type = 'user_activity' 
        AND recommendation_type = 'games' THEN
      DELETE
    """

    # Execute the SQL statement
    query_job = client.query(merge_sql)
    
manage_down_stream_update()

Content-based recommendations similar to 'airplane survival':
Content-based recommendations similar to 'high school gossip':
Content-based recommendations similar to 'santa airline':
Content-based recommendations similar to 'connectricity':
Content-based recommendations similar to 'megacity hop':
Content-based recommendations similar to 'happy crayon':
Content-based recommendations similar to 'math bat':
Content-based recommendations similar to 'kill spy':
Content-based recommendations similar to 'classic solitaire':
Content-based recommendations similar to 'air hockey':
Content-based recommendations similar to 'dream body workout':
Content-based recommendations similar to 'tripoly':
Content-based recommendations similar to 'oddbods monster truck driving':
Content-based recommendations similar to 'extreme baseball':
Content-based recommendations similar to 'teddy bubble rescue':
Content-based recommendations similar to 'drone pickup service':
Content-based recommendations similar to 'c

/var/folders/__/7g992vcd0fs5t43310f6sl8m0000gn/T/ipykernel_5839/457276169.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_user_games_recommendations['ranking'].fillna(-1, inplace=True)


Dropped table bright-arc-328707.ayoba.rec_games_recommendations_activity_staging
Created table bright-arc-328707.ayoba.rec_games_recommendations_activity_staging


100%|██████████| 1/1 [00:00<00:00, 3226.39it/s]


Data loading done
